In [ ]:
import os
import sys

curruser = os.environ.get('USER')



# sys.path.insert(0, './../src/')
sys.path.insert(0, '/home/{}/notebooks/support_library/'.format(curruser)) 
sys.path.insert(0, '/home/{}/python35-libs/lib/python3.5/site-packages/'.format(curruser))
sys.path.insert(0, '/home/{}/notebooks/labdata/lib/'.format(curruser))

from pathlib import Path
import re
import pandas as pd
import numpy as np
from tqdm import tqdm_notebook
import utils
from Bio import SeqIO
from sklearn.preprocessing import LabelEncoder

In [ ]:
dnaSegPath = ('./../data/dnaseg/mart_export.txt')

In [ ]:
def streamFastaIO (path):
    patt = re.compile(r'^[^N][ATCG]+')
    for sequence in SeqIO.parse(path,"fasta"):
        outSeqStr = str(sequence.seq)
        if len(patt.findall(outSeqStr))!=0:
            tup = (outSeqStr, len(sequence))
        else:
            tup = ('', -1)
        yield tup

In [ ]:
lenc = LabelEncoder()

In [ ]:
lenc.fit_transform(np.array(['A','T','C','G']))

In [ ]:
def getChunkesFromSeqToRight(seq, right_len=30, minlen = 5, lenc=None):
    i=minlen
    chunkstuple = []
    while True:
        if len(seq)-i >= right_len:
            left_chunk = seq[:i]
            if len(left_chunk) > right_len: 
                left_chunk = left_chunk[-right_len:] 
            chunkstuple += [(' '.join([left_chunk, seq[i+1:i+right_len+1]]), lenc.transform([seq[i]])[0])]
            i+=1
        else:
            chunkstuple += [(' '.join([left_chunk, seq[i+1:len(seq)+1]]), lenc.transform([seq[i]])[0])]
            break
    return chunkstuple

In [ ]:
def getChunkesFromSeqToLeft(seq, left_len=30, minlen = 5, lenc = None):
    i = minlen
    chunkstuple = []
    while True:
        if len(seq)-i >= left_len:
            right_chunk = seq[-i:]
            if len(right_chunk) > left_len: 
                right_chunk = right_chunk[:left_len] 
            chunkstuple += [(' '.join([seq[-(i+left_len+1):-(i+1):], right_chunk]), lenc.transform([seq[-(i+1)]])[0])]
            i+=1
        else:
            chunkstuple += [(' '.join([seq[-len(seq):-(i+1):], right_chunk]), lenc.transform([seq[-(i+1)]])[0])]
            break
    return chunkstuple

In [ ]:
seqArr = []
for seq, _len in tqdm_notebook(streamFastaIO(dnaSegPath)):
     seqArr+=[(seq, _len)]

In [ ]:
import csv
csvpath = Path.joinpath(Path(os.getcwd()),'../data/dnaseg/csv','DnaSeg4Train.csv')

In [ ]:
finalarr = []
headers = ['seq','label']
for item in tqdm_notebook(seqArr):
    if item[1] != -1:
        chunkesRight = getChunkesFromSeqToRight(item[0], lenc = lenc)
        chunkesLeft  = getChunkesFromSeqToLeft(item[0],  lenc = lenc)
        res = chunkesRight+chunkesLeft
        finalarr+=res

        with open(str(csvpath),'a', encoding='utf8') as outcsv: 
            writer = csv.writer(outcsv,delimiter=';', quotechar='"', lineterminator='\n')
            file_is_empty = os.stat(str(csvpath)).st_size ==0
            if file_is_empty:
                writer.writerow(headers)
            for item in finalarr:
                writer.writerow(item)
    else:
        pass